In [11]:
import numpy as np
import keras

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings

warnings.filterwarnings("ignore")

from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import log_loss
from keras import __version__ as keras_version
from keras.callbacks import EarlyStopping
import load_images
from sklearn.model_selection import StratifiedKFold
import data_set
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
keras_version

'2.0.1'

In [4]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [12]:
WIDTH = 32
HEIGHT = 32
BATCH_SIZE =16

In [5]:
FLAGS = dict()
FLAGS['width'] = 32
FLAGS['height'] = 32
FLAGS['batch_size'] = 20
FLAGS['kernel_1_out'] = 8
FLAGS['kernel_2_out'] = 8
FLAGS['conv2_input_width'] = 16
FLAGS['conv2_input_height'] = 16
FLAGS['n_classes'] = 8
FLAGS['learning_rate'] = 0.001
FLAGS['batch_size'] = 16
FLAGS['n_epochs'] = 20
FLAGS['train_report_step'] = 20
FLAGS['val_report_step'] = 80
FLAGS['keep_prob'] = 0.75
FLAGS['reg'] = 0.01
FLAGS['patience'] = 3

In [9]:
X,y = load_images.load_train(WIDTH, HEIGHT)

Read train images
Load folder ALB (Index: 0)
Load folder BET (Index: 1)
Load folder DOL (Index: 2)
Load folder LAG (Index: 3)
Load folder NoF (Index: 4)
Load folder OTHER (Index: 5)
Load folder SHARK (Index: 6)
Load folder YFT (Index: 7)
Read train data time: 416.44 seconds


In [19]:
y_un = np.argmax(y, axis=1) 

In [21]:
y_un.shape

(3777,)

In [13]:
def create_model():
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(32, 32, 3)))
    model.add(Convolution2D(4, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(4, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(8, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(8, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='softmax'))

    sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')

    return model


In [27]:
model.fit?

In [30]:
skf = StratifiedKFold(n_splits=5)

split = 0
best_val_loss = 10000
best_model = None
for train_i, val_i in skf.split(X, y_un):
    print("split {}".format(split))
    print('======================')
    split +=1
    model = create_model()
    X_train, X_val = X[train_i], X[val_i]
    y_train, y_val = y[train_i], y[val_i]
    
    callbacks = [
            EarlyStopping(monitor='val_loss', patience=1, verbose=0)]
    
    model.fit(x=X_train, 
              y=y_train, 
              shuffle=True, 
              validation_data=(X_val, y_val), 
              callbacks=callbacks, 
              batch_size=BATCH_SIZE, 
              epochs=20)
    
    val_loss = model.evaluate(x=X_val, y=y_val, batch_size=BATCH_SIZE)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
    
    

split 0
Train on 3019 samples, validate on 758 samples
Epoch 1/20
3019/3019 [==============================] - 16s - loss: 1.6943 - val_loss: 1.6059
Epoch 2/20
3019/3019 [==============================] - 13s - loss: 1.6099 - val_loss: 1.5859
Epoch 3/20
3019/3019 [==============================] - 12s - loss: 1.5980 - val_loss: 1.5554
Epoch 4/20
3019/3019 [==============================] - 15s - loss: 1.5681 - val_loss: 1.5025
Epoch 5/20
3019/3019 [==============================] - 16s - loss: 1.5343 - val_loss: 1.4945
Epoch 6/20
3019/3019 [==============================] - 12s - loss: 1.5058 - val_loss: 1.4282
Epoch 7/20
3019/3019 [==============================] - 14s - loss: 1.4431 - val_loss: 1.3640
Epoch 8/20
3019/3019 [==============================] - 14s - loss: 1.4445 - val_loss: 1.3499
Epoch 9/20
3019/3019 [==============================] - 12s - loss: 1.4860 - val_loss: 1.4158
Epoch 10/20
758/758 [==============================] - 1s     
split 1
Train on 3020 samples, valid

In [33]:
import load_images
X_test, y_test = load_images.load_test(width = FLAGS['width'], 
                                                      height=FLAGS['height'])

loading 0 of 1000
loading 20 of 1000
loading 40 of 1000
loading 60 of 1000
loading 80 of 1000
loading 100 of 1000
loading 120 of 1000
loading 140 of 1000
loading 160 of 1000
loading 180 of 1000
loading 200 of 1000
loading 220 of 1000
loading 240 of 1000
loading 260 of 1000
loading 280 of 1000
loading 300 of 1000
loading 320 of 1000
loading 340 of 1000
loading 360 of 1000
loading 380 of 1000
loading 400 of 1000
loading 420 of 1000
loading 440 of 1000
loading 460 of 1000
loading 480 of 1000
loading 500 of 1000
loading 520 of 1000
loading 540 of 1000
loading 560 of 1000
loading 580 of 1000
loading 600 of 1000
loading 620 of 1000
loading 640 of 1000
loading 660 of 1000
loading 680 of 1000
loading 700 of 1000
loading 720 of 1000
loading 740 of 1000
loading 760 of 1000
loading 780 of 1000
loading 800 of 1000
loading 820 of 1000
loading 840 of 1000
loading 860 of 1000
loading 880 of 1000
loading 900 of 1000
loading 920 of 1000
loading 940 of 1000
loading 960 of 1000
loading 980 of 1000


In [40]:
preds = best_model.predict(X_test)

In [41]:
import make_submission
submit = make_submission.makeSubmission(preds, y_test)

In [42]:
submit.tail(20)

,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT,image
980,0.138543,0.068164,1.037430e-01,1.685805e-02,0.106028,3.903075e-02,1.235360e-02,0.515280,img_07663.jpg
981,0.419951,0.069790,3.913227e-02,2.770817e-02,0.104271,1.284306e-01,3.509553e-02,0.175621,img_07678.jpg
982,0.554872,0.054608,2.218880e-02,1.049342e-02,0.100533,8.660426e-02,1.340046e-02,0.157301,img_07689.jpg
983,0.419951,0.069790,3.913227e-02,2.770817e-02,0.104271,1.284306e-01,3.509553e-02,0.175621,img_07700.jpg
984,0.989740,0.000265,9.137295e-07,8.423283e-09,0.000786,1.176657e-03,1.034016e-06,0.008031,img_07717.jpg
985,0.429832,0.066134,3.763628e-02,2.244033e-02,0.102001,1.175126e-01,3.862167e-02,0.185823,img_07746.jpg
986,0.713854,0.034809,1.433796e-02,3.867759e-03,0.065899,5.776533e-02,3.229021e-03,0.106237,img_07757.jpg
987,0.462357,0.061241,3.282772e-02,1.682085e-02,0.099077,1.054857e-01,3.471546e-02,0.187475,img_07792.jpg
988,0.419951,0.069790,3.913227e-02,2.770817e-02,0.104271,1.284306e-01,3.509553e-02,0.175621,img_07799.jpg
989,0.984755,0.001467,1.451295e-04,6.793831e-06,0.007546,2.159228e-03,8.991510e-08,0.003921,img_07818.jpg


In [43]:
submit.to_csv('k_fold.csv', index=False)